In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Define App and read data
app = dash.Dash(__name__)
df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
)

year_list = [i for i in range(1980, 2024, 1)]

# App skeleton

app.layout = html.Div(
    children=[
        # Title
        html.H1(
            "Automobile Sales Statistics Dashboard",
            style={
                "color": "#503D36",
                "font-size": "24",
                "textAlign": "center",
            },
        ),
        # Dropdown menu 1
        html.Div(
            children=[
                html.Label("Select Statistics:"),
                dcc.Dropdown(
                    id="dropdown-statistics",
                    options=[
                        {"label": "Yearly Statistics", "value": "Yearly Statistics"},
                        {"label": "Recession Period Statistics","value": "Recession Period Statistics",},
                    ],
                    placeholder="Select a report type",
                    value="Select Statistics",
                    style={
                        "width": "80%",
                        "padding": "3px",
                        "font-size": "20px",
                        "text-align": "center",
                    },
                ),
            ]
        ),
        # Dropdown menu 2
        html.Div(
            children=[
                dcc.Dropdown(
                    id="select-year",
                    options=[{"label": i, "value": i} for i in year_list],
                    value="Select year",
                    placeholder="Select year",
                    style={
                        "width": "80%",
                        "padding": "3px",
                        "font-size": "20px",
                        "text-align": "center",
                    },
                )
            ]
        ),
        html.Div(
            children=[
                html.Div(
                    id="output-container",
                    className="chart-grid",
                    style={"display": "flex"},
                )
            ]
        ),
    ],
    style={"backgroundColor": "white"},
)


@app.callback(
    Output(component_id="select-year", component_property="disabled"),
    Input(component_id="dropdown-statistics", component_property="value"),
)
def update_input_container(selected_statistics):
    if selected_statistics == "Yearly Statistics":
        return False
    else:
        return True


@app.callback(
    Output(component_id="output-container", component_property="children"),
    [
        Input(component_id="select-year", component_property="value"),
        Input(component_id="dropdown-statistics", component_property="value"),
    ],
)
def update_output_container(input_year, selected_statistics):
    if selected_statistics == "Recession Period Statistics":
        # Filter the data for recession periods
        recession_data = df[df["Recession"] == 1]

        # Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        # use groupby to create relevant data for plotting
        yearly_rec = (
            recession_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        )
        R_chart1 = dcc.Graph(
            figure=px.line(
                yearly_rec,
                x="Year",
                y="Automobile_Sales",
                title="Average Automobile Sales fluctuation over Recession Period",
            )
        )

        # Plot 2 Calculate the average number of vehicles sold by vehicle type
        # use groupby to create relevant data for plotting
        average_sales = (
            recession_data.groupby("Vehicle_Type")["Automobile_Sales"]
            .mean()
            .reset_index()
        )
        R_chart2 = dcc.Graph(
            figure=px.bar(
                average_sales,
                x="Vehicle_Type",
                y="Automobile_Sales",
                title="Average Number Of Vehicles Sold By Vehicle Type",
            )
        )

        # Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        # use groupby to create relevant data for plotting
        exp_rec = (
            recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .sum()
            .reset_index()
        )
        R_chart3 = dcc.Graph(
            figure=px.pie(
                exp_rec,
                values="Advertising_Expenditure",
                names="Vehicle_Type",
                title="Total Expenditure Share By Vehicle Type During Recessions",
            )
        )

        # Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_rec = (
            recession_data.groupby("Vehicle_Type")[
                ["unemployment_rate", "Automobile_Sales"]
            ]
            .mean()
            .reset_index()
        )
        R_chart4 = dcc.Graph(
            figure=px.bar(
                unemp_rec,
                x="unemployment_rate",
                y="Automobile_Sales",
                color="Vehicle_Type",
                title="Effect Of Unemployment Rate On Vehicle Type And Sales",
            )
        )

        return [
            html.Div(
                className="chart-item",
                children=[html.Div(children=R_chart1), html.Div(children=R_chart2)],
                style={"display": "flex"},
            ),
            html.Br(),
            html.Div(
                className="chart-item",
                children=[html.Div(children=R_chart3), html.Div(children=R_chart4)],
                style={"display": "flex"},
            ),
        ]

    elif input_year and selected_statistics == "Yearly Statistics":
        yearly_data = df[df["Year"] == input_year]

        # TASK 2.5: Creating Graphs Yearly data

        # plot 1 Yearly Automobile sales using line chart for the whole period.
        yas = df.groupby("Year")["Automobile_Sales"].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(
                yas, x="Year", y="Automobile_Sales", title="Yearly Automobile Sales"
            )
        )

        # Plot 2 Total Monthly Automobile sales using line chart.
        monthly_as = df.groupby("Month")["Automobile_Sales"].mean().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(
                monthly_as,
                x="Month",
                y="Automobile_Sales",
                title="Total Monthly Automobile Sales",
            )
        )

        # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata = (
            yearly_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        )
        Y_chart3 = dcc.Graph(
            figure=px.bar(
                avr_vdata,
                x="Vehicle_Type",
                y="Automobile_Sales",
                title="Average Vehicles Sold by Vehicle Type in the year {}".format(
                    input_year
                ),
            )
        )

        # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = (
            yearly_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .sum()
            .reset_index()
        )
        Y_chart4 = dcc.Graph(
            figure=px.pie(
                exp_data,
                values="Advertising_Expenditure",
                names="Vehicle_Type",
                title="Total Advertisement Expenditure for each Vehicle",
            )
        )

        # TASK 2.6: Returning the graphs for displaying Yearly data
        return [
            html.Div(
                className="chart-item",
                children=[html.Div(Y_chart1), html.Div(Y_chart2)],
                style={"display": "flex"},
            ),
            html.Br(),
            html.Div(
                className="chart-item",
                children=[html.Div(Y_chart3), html.Div(Y_chart4)],
                style={"display": "flex"},
            ),
        ]

    else:
        return None


if __name__ == "__main__":
    app.run_server(port=8080)